In [1]:
#imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN 5110)


In [16]:
# network config
input_size = 256
filter1_size = 32
filter2_size = 64 
classes = 8
batch_size = 32
epochs=10

In [17]:
def keras_model_conv():
    model = Sequential()
    model.add(Convolution2D(filter1_size, 3, 3, 
                          border_mode='same', 
                          input_shape=(3, input_size, input_size)))
    model.add(Activation('relu'))
    model.add(Convolution2D(filter1_size, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(filter2_size, 3, 3))
    model.add(Activation('relu'))
    model.add(Convolution2D(filter2_size, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

    return model


In [18]:
datagen = ImageDataGenerator(horizontal_flip=True,
                            rescale=1./255.,
                            rotation_range=10)
gen = datagen.flow_from_directory("../Data/train",
                            target_size=(input_size,input_size),
                           batch_size=batch_size,
                           save_to_dir="../Data/augmented")



Found 3777 images belonging to 8 classes.


In [ ]:
model = keras_model_conv()
history = model.fit_generator(gen,
                              samples_per_epoch=1000,
                              nb_epoch=epochs,
                              verbose=1 
                              )
score = model.evaluate(X_test, Y_test, verbose=0)


Epoch 1/10
1024/1000 [==============================] - 86s - loss: 2.3364 - acc: 0.4307      

D:\Anaconda2\lib\site-packages\keras\engine\training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Epoch 2/10
1024/1000 [==============================] - 84s - loss: 1.6558 - acc: 0.4502     